<a href="https://colab.research.google.com/github/PnrSvc/Bert-Examples/blob/main/bert-example-en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# İmport Library

In [1]:
!pip install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio===0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install transformers requests beautifulsoup4 pandas numpy
from transformers import AutoTokenizer,AutoModelForSequenceClassification,pipeline
import torch
import requests
from bs4 import BeautifulSoup
import re 
import numpy as np
import pandas as pd

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.6 MB/s eta 0:10:36tcmalloc: large alloc 1147494400 bytes == 0x55ac30640000 @  0x7f7de743e615 0x55abf78803bc 0x55abf796118a 0x55abf78831cd 0x55abf7975b3d 0x55abf78f7458 0x55abf78f202f 0x55abf7884aba 0x55abf78f72c0 0x55abf78f202f 0x55abf7884aba 0x55abf78f3cd4 0x55abf7976986 0x55abf78f3350 0x55abf7976986 0x55abf78f3350 0x55abf7976986 0x55abf78f3350 0x55abf7884f19 0x55abf78c8a79 0x55abf7883b32 0x55abf78f71dd 0x55abf78f202f 0x55abf7884aba 0x55abf78f3cd4 0x55abf78f202f 0x55abf7884aba 0x55abf78f2eae 0x55abf78849da 0x55abf78f3108 0x55abf78f202f
     |██████████████████▌             | 1055.7 MB 1.7 MB/s eta 0:07:22tcmalloc: large alloc 1434370048 bytes == 0x55ac74c96000 @  0x7f7de743e615 0x55abf78803bc 0x55abf796118a 0x55abf78831cd 0x55abf7975b3d 0x55abf78f7458 0x55abf78f202f 0x55abf7884aba 0x55abf78f72c0 0x55abf78f202f 0x55abf7884aba 0x55abf78f3cd4 0x55abf7976986 0x55ab

# Bert Model İmplementation

In [2]:
tokenizer=AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model=AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [3]:
tokens= tokenizer.encode('A lovely big restaurant',return_tensors='pt')

In [4]:
result=model(tokens)

In [5]:
result.logits

tensor([[-2.6103, -2.4793, -0.3228,  1.9497,  2.6809]],
       grad_fn=<AddmmBackward0>)

In [6]:
int(torch.argmax(result.logits))+1

5

# Web-Scraping

In [7]:
r=requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup=BeautifulSoup(r.text,'html.parser')
regex=re.compile('.*comment.*')
result=soup.find_all('p',{'class':regex})
reviews=[result.text for result in result]

In [8]:
reviews

['The food is fresh and tasty. \xa0The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.',
 "Don't come here expecting legit Mexican food but a modern twist on some staples. Loud party area, fun drinks and friendly staff make this a hip meeting area for large groups. Drinks were better than the food. They stuff the families toward the back but lack any amenities (no changing table) except a high chair. Service started off friendly but it took a while to get someone to take our order and then they forgot our dish which came out cold when we asked for it. Then we had to flag someo

In [9]:
df=pd.DataFrame(np.array(reviews), columns=['review'])

In [10]:
df.head()

,review
0,The food is fresh and tasty. The scallop cevi...
1,Don't come here expecting legit Mexican food b...
2,Out of all the restaurants that I tried in Syd...
3,We came here on a Thursday night @ 5pm and by ...
4,I was pleasantly surprised at what a great job...


In [11]:
df['review'].iloc[0]

'The food is fresh and tasty. \xa0The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.'

In [12]:
def sentiment_score(review):
  tokens= tokenizer.encode(review,return_tensors='pt')
  result=model(tokens)
  return int(torch.argmax(result.logits))+1

In [13]:
sentiment_score(df['review'].iloc[0])

4

In [14]:
df['sentiment']=df['review'].apply(lambda x: sentiment_score(x[:512]))

In [15]:
df

,review,sentiment
0,The food is fresh and tasty. The scallop cevi...,4
1,Don't come here expecting legit Mexican food b...,3
2,Out of all the restaurants that I tried in Syd...,5
3,We came here on a Thursday night @ 5pm and by ...,4
4,I was pleasantly surprised at what a great job...,5
5,Really nice (upmarket) Mexican restaurant. Goo...,4
6,Ordered feed me for $59 along with that.. Food...,2
7,"The best tasting cauliflower taco, mushroom ce...",5
8,Have been here twice and have absolutely loved...,5
9,If you're looking for a quiet little romantic ...,2


In [16]:
df['review'].iloc[3]

'We came here on a Thursday night @ 5pm and by 6pm the place was packed. A lovely big restaurant with a bar at the front (which is a bit awkward to try and push past everyone to get to your table). Friendly, helpful staff which is always a good start. The menu is large so we went with the "feed me" selection. All you need to do is sit back and let the chef feed you. As the other reviewers have stated the corn is a highlight and the pulled pork tacos, the sangria wasn\'t bad either.Loved the Mexican tapas style food and will be back.'